# 📌 형태소 분석

### ✏️ 파일 읽기

In [ ]:
import os

# 파일 읽기
def read_txt_file(file_name):
  print("[파일명] " + file_name)
  text = ""
  
  # 파일 열기
  with open(file_path + file_name, "r", encoding="UTF8") as txt_file:
    # 파일 라인별로 텍스트만 추출
    for line in txt_file:
      line = line.strip()
      text += line
  return text

file_path = "C:\\Users\\sodud\\study\\ssafy_spec_pjt\\subtitle\\data\\refined_text\\"

# 자막 파일 폴더
file_list = os.listdir(file_path)

# 자막 파일 목록
txt_files = [txt_file for txt_file in file_list]

for txt_file in txt_files:
  read_txt_file(txt_file)

### ✏️ 불용어 제거
* 불용어 리스트 직접 설정  
* nltk에 파일 생성 후 라이브러리 사용 (✔️)

In [ ]:
import re

# 불용어 리스트 생성
stop_words = []
with open("stop_words_japanese.txt", "r", encoding="UTF8") as f :
  lines = f.readlines()
  # for stop_word in lines:
  #   stop_words.append(stop_word.strip())
  stop_words = [stop_word.strip() for stop_word in lines]

# 생성된 불용어 리스트 확인
print("불용어: ", stop_words)

In [ ]:
import nltk
nltk.download('stopwords')

### ✏️ 사전 정보 확인

In [2]:
import MeCab

try:
  # 품사 분석
  t = MeCab.Tagger()
  
  # 일본어 사전 정보 출력
  d = t.dictionary_info()
  while d:
    print("filename: %s" % d.filename)
    print("charset: %s" %  d.charset)
    print("size: %d" %  d.size)
    print("type: %d" %  d.type)
    print("lsize: %d" %  d.lsize)
    print("rsize: %d" %  d.rsize)
    print("version: %d" %  d.version)
    d = d.next


except RuntimeError as e:
  print("RuntimeError:", e)

filename: C:\Users\sodud\AppData\Roaming\Python\Python39\site-packages\unidic_lite\dicdir\sys.dic
charset: utf8
size: 756264
type: 0
lsize: 5981
rsize: 5981
version: 102


In [ ]:
import sys
import MeCab
import neologdn
from nltk.probability import FreqDist
from matplotlib import rcParams

rcParams["font.family"] = 'sans-serif'
  
# 포함 품사: 명사, 동사, 형용사, 부사
CONTENT_WORD_POS = ("名詞", "動詞", "形容詞", "副詞")
# 제외 품사: 접미사, 비자립, 대명사, 미지어
IGNORE = ("接尾", "非自立", "代名詞", "未知語")

def is_content_word(feature):
  return feature.startswith(CONTENT_WORD_POS) and all(f not in IGNORE for f in feature.split(",")[:7])

sentence = ""
file_path = "C:\\Users\\sodud\\study\\ssafy_spec_pjt\\subtitle\\data\\refined_text\\"
text = open(file_path + "Getter_Robot_Go_001.txt", "r", encoding="UTF8")
for line in text:
  sentence += line

try:
  # 품사 분석
  tagger = MeCab.Tagger()
  
  # 기본 출력 결과
  # print(tagger.parse(sentence))
  
  sentence = neologdn.normalize(sentence,repeat = 2)
  sentence = "".join([i for i in sentence if i.isalpha() or i.isspace()])
  result = tagger.parseToNode(sentence)
  
  stop_words = []
  with open("stop_words_japanese.txt", "r", encoding="UTF8") as f :
    lines = f.readlines()
    stop_words = [stop_word.strip() for stop_word in lines]
    
  content_words = []
  
  while result:
    if is_content_word(result.feature):
      lemma = result.feature.split(",")[7] if len(result.feature.split(",")) > 7 and result.feature.split(",")[7] != "*" else result.surface
      if lemma not in stop_words:
        content_words.append(lemma.split("-")[0])
    result = result.next
  
  print(content_words)
  
  fdist = FreqDist(content_words)
  fdist.plot(10, cumulative=False)
  
except RuntimeError as e:
    print("RuntimeError:", e)